In [1]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    model_name="llama-3.1-8b-instant",
    api_key="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",
)

In [2]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    template="""<|begin_of_text|>
    <|begin_of_text|>
    <|start_header_id|>system<|end_header_id|>  
    You are an expert at identifying table of contents from the given paragraph {paragraph} 
    Don't give any additional information or summary
    You must add the sub category but don't add 'The Official Gazette' and which edition

    ### The output must be in the following format: 
    List of dictonary, each dictonary key should be main topics like Executive Council Resolutions, Laws and value should be list of sub topics
    Do not include Pages

    Sample output format
    
    {{
        "Laws": [
            "Law No. (2) of 2022 Concerning the Reorganisation of the National Rehabilitation Centre - Abu Dhabi"
        ],
        "Executive Council Resolutions": [
            "Chairman of the Executive Council Resolution No. (3) of 2022 Concerning the Appointment of the Director-General for Tourism at the Department of Culture and Tourism"
        ],
        "Circulars": [
            "Circular No. (1) of 2022 Concerning the Policy of Providing an Easy and Effortless Customer Experience in the Emirate of Abu Dhabi"
        ],
            "Decrees": [
            "Amiri Decree No. (13) of 2023 Concerning Sending a Judge to Retirement"
        ]
    }}
    
    <|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    """
)

question_router = prompt | llm


In [3]:
import os
import re
import json
from PyPDF2 import PdfReader

def remove_dot(text):
    return text[:-1] if text.endswith('.') else text

def extrating_content(key_topics, output_lower, final_output):

    for i in range(len(key_topics)):
        current_topic = key_topics[i]
        
        current_topic = remove_dot(current_topic)
        current_topic_lower = current_topic.lower()

        print("===========================================")
        current_topic = current_topic_lower.replace("'", '"')
        
        print(f"Current Topic: {current_topic}")

        if i < len(key_topics) - 1:
            next_topic = key_topics[i + 1]
            next_topic = remove_dot(next_topic)
            next_topic_lower = next_topic.lower()
            
            next_topic = next_topic_lower.replace("'", '"')
            print(f"Next Topic: {next_topic}")

            start = output_lower.find(current_topic)
            end = output_lower.find(next_topic)
            
            print(f"String index of current topic: {start}")
            print(f"String index of next topic: {end}")

            substring = output_lower[start:end]
        else:
            # For the last topic, take everything from the current topic to the end
            start = output_lower.find(current_topic) + len(current_topic)
            end = len(output_lower)  # Go to the end of the string
            substring = output_lower[start:end]

        print("\n")
        print(substring)

        final_output[current_topic] = substring
    
    return final_output

def table_of_content(reader):
    for page_no in range(1, 4):
        page = reader.pages[page_no]
        text = page.extract_text()
        return text

def parse_document(pdf_path):
    with open(pdf_path, 'rb') as file:

        key_topics = list()

        reader = PdfReader(file)
        num_pages = len(reader.pages)

        index_text = table_of_content(reader)
        index_data = question_router.invoke({"paragraph": index_text})
        output = ""
        for page_no in range(3, num_pages):
            page = reader.pages[page_no]
            text = page.extract_text()
            if text and text[0].isdigit():
                text = text[1:]
                text = re.sub(r'^\d+', '', text)

            cleaned_text = re.sub(r'[\n\r\t]', ' ', text)
            cleaned_text = cleaned_text.replace("  ", " ")
            cleaned_text = cleaned_text.replace("  ", " ")

            output = output + " " + cleaned_text
            output = output.strip()
        
        print("TOC\n")
        print(index_data.content)
        toc = json.loads(index_data.content)

        for key, value in toc.items():
            key_topics.extend(value)
        
        return toc, key_topics, output

In [4]:


directory_path = '../doc/test/'
final_output ={}

for filename in os.listdir(directory_path):
    if filename.endswith('.pdf'): 
        file_path = os.path.join(directory_path, filename) 
        print(f"Reading {file_path}...")
        
        toc, key_topics, output = parse_document(file_path)

        final_output = extrating_content(key_topics, output.lower(), final_output)
        

Reading ../doc/test/2 nd Edition English 2023.pdf...
TOC

{
    "Executive Council Resolutions": [
        "Executive Council Resolution No. (11) of 2023 Concerning The Reformation of the Board of Trustees of Mohamed bin Zayed University for Humanities",
        "Executive Council Resolution No. (12) of 2023 concerning The Reformation of the Board of Directors of Abu Dhabi Housing Authority",
        "Executive Council Resolution No. (13) of 2023 concerning the Appointment of the Director-General of Abu Dhabi Housing Authority",
        "Executive Council Resolution No. (15) of 2023 concerning the Appointment of the Director of Mohamed bin Zayed University for Humanities"
    ]
}
Current Topic: executive council resolution no. (11) of 2023 concerning the reformation of the board of trustees of mohamed bin zayed university for humanities
Next Topic: executive council resolution no. (12) of 2023 concerning the reformation of the board of directors of abu dhabi housing authority
String in

In [6]:
len(final_output)

155

In [11]:
final_output

{'law no. (12) of 2021 concerning the amendment of some provisions of law no. (28) of 2007 concerning the reorganisation of the zoo and aquarium public institution in al ain': 'law no. (12) of 2021 concerning the amendment of some provisions of law no. (28) of 2007 concerning the reorganisation of the zoo and aquarium public institution in al ain we, khalifa bin zayed al nahyan, ruler of abu dhabi having reviewed: • law no. (1) of 1974 concerning the reorganisation of the governmental body in the emirate of abu dhabi and its amendments; • law no. (2) of 1971 concerning the national advisory council, and its amendments; • law no. (28) of 2007 concerning the reorganisation of the zoo and aquarium public institution in al ain, and its amendments; • law no. (8) of 2018 concerning the establishment of the department of culture and tourism; • chairman of the executive council resolution no. (23) of 2018 concerning the additional competencies of the department of culture and tourism; and • ba

In [16]:
import json

# Specify the filename
filename = 'data.json'

# # Write the dictionary to a JSON file
# with open(filename, 'w') as json_file:
#     json.dump(final_output, json_file, indent=4)  # indent for pretty printing

# print(f'Dictionary saved to {filename}')

with open(filename, 'r') as json_file:
    loaded_data = json.load(json_file)

In [87]:
type(loaded_data)

dict

`Extracting Law reference`

In [71]:
prompt = PromptTemplate(
        template="""
        <|begin_of_text|>
        <|start_header_id|>system<|end_header_id|>  
        You are an expert to identifying laws from the given legal document paragraph. 

        {paragraph}

        ## Instructions:
        1. Just return output only.
        2. make as short summary of each key phrase and don't elaburate much.
        3. Do not include sequence numbers
        4. Output format must be list of laws like ["Law1", "Law2", .,]

        <|eot_id|>
        <|start_header_id|>assistant<|end_header_id|>
        """,
        input_variables=["paragraph"],
    )

def key_phrase_extraction(paragraph, prompt, llm):
    question_router = prompt | llm
    response = question_router.invoke({"paragraph": paragraph})
    return response.content

In [73]:
for key, value in loaded_data.items():
    print(f"{key}\n")
    reference = key_phrase_extraction(value, prompt, llm)
    print(f"Reference: {reference}")


law no. (12) of 2021 concerning the amendment of some provisions of law no. (28) of 2007 concerning the reorganisation of the zoo and aquarium public institution in al ain

Reference: ["Law No. (1) of 1974", "Law No. (2) of 1971", "Law No. (28) of 2007", "Law No. (8) of 2018", "Chairman of the Executive Council Resolution No. (23) of 2018"]
law no. (13) of 2021 concerning the amendment of some provisions of law no. (5) of 1992 concerning the establishment of zayed bin sultan al nahyan charitable and humanitarian foundation

Reference: Law No. (1) of 1974 concerning the reorganisation of the governmental body in the emirate of Abu Dhabi and its amendments
Law No. (2) of 1971 concerning the national advisory council, and its amendments
Law No. (5) of 1992 concerning Zayed Bin Sultan Al Nahyan Charitable and Humanitarian Foundation
Law No. (13) of 2021 concerning the amendment of some provisions of Law No. (5) of 1992
amiri decree no. (8) of 2020 concerning sending a member of a judicial 

APIStatusError: Error code: 413 - {'error': {'message': 'Request too large for model `llama-3.1-8b-instant` in organization `org_01j2jwgjmaebgr4rjbw0by2feh` on tokens per minute (TPM): Limit 20000, Requested 35683, please reduce your message size and try again. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

In [80]:
from typing import Optional, List, Dict
from pydantic import BaseModel, Field
from langchain_groq import ChatGroq

llm1 = ChatGroq(
    temperature=0,
    model_name="llama-3.1-8b-instant",
    api_key="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",
)


In [86]:
class Reference(BaseModel):
    Laws: Optional[List[str]] = Field(default_factory=dict, description="List of Laws")


prompt = PromptTemplate(
    template="""<|begin_of_text|>
    <|start_header_id|>system<|end_header_id|>  
    You are an expert in identifying laws from the given legal document paragraph. 

    {paragraph}

    Provide a short summary of each key phrase as a list of laws without elaboration.
    Output format:
    Laws: ["Law No. (1)", "Law No. (2)", ...]  # Make sure to follow this format exactly

    <|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["paragraph"],
)

structured_llm = llm1.with_structured_output(Reference)
question_router = prompt | structured_llm

paragraph = final_output["law no. (16) of 2022 concerning the licensing of the drivers and vehicles in the emirate of abu dhabi"]
# paragraph = final_output["law no. (1) of 2023 concerning the establishment of abu dhabi transport company “public joint stock company”"]
response = question_router.invoke({"paragraph": paragraph})
response

BadRequestError: Error code: 400 - {'error': {'message': "Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': ' {"Laws": ["Law No. (1) of 1974", "Law No. (2) of 1971", "Law No. (19) of 2006", "Law No. (30) of 2019", "Law No. (1) of 2020", "Federal Law No. (21) of 1995"]}'}}

In [26]:
print(response)

Laws=['Law No. (16) of 2022 concerning the licensing of drivers and vehicles in the Emirate of Abu Dhabi', 'Law No. (1) of 1974 concerning the reorganisation of the governmental body in the Emirate of Abu Dhabi', 'Law No. (2) of 1971 concerning the national advisory council', 'Law No. (19) of 2006 concerning the regulation of transportation via taxis in the Emirate of Abu Dhabi', 'Law No. (30) of 2019 concerning the establishment of the Department of Municipalities and Transport', 'Law No. (1) of 2020 concerning the General Headquarters of Abu Dhabi Police'] Decrees=[] Circulars=[] Executive_Council_Resolutions=[] Crown_Prince_Chairman_of_the_Executive_Council_Resolutions=[]


In [29]:
response.Laws

['Law No. (16) of 2022 concerning the licensing of drivers and vehicles in the Emirate of Abu Dhabi',
 'Law No. (1) of 1974 concerning the reorganisation of the governmental body in the Emirate of Abu Dhabi',
 'Law No. (2) of 1971 concerning the national advisory council',
 'Law No. (19) of 2006 concerning the regulation of transportation via taxis in the Emirate of Abu Dhabi',
 'Law No. (30) of 2019 concerning the establishment of the Department of Municipalities and Transport',
 'Law No. (1) of 2020 concerning the General Headquarters of Abu Dhabi Police']

In [31]:
print(paragraph)

law no. (16) of 2022 concerning the licensing of the drivers and vehicles in the emirate of abu dhabi we, mohamed bin zayed al nahyan, ruler of abu dhabi, having reviewed: • law no. (1) of 1974 concerning the reorganisation of the governmental body in the emirate of abu dhabi and its amendments; • law no. (2) of 1971 concerning the national advisory council and its amendments; • law no. (19) of 2006 concerning the regulation of tansportation via taxis in the emirate of abu dhabi and its amendments; • law no. (30) of 2019 concerning the establishment of the department of municipalities and transport; • law no. (1) of 2020 concerning the general headquarters of abu dhabi police, and its amendments; • federal law no. (21) of 1995 concerning the traffic, and its amendments; and • based on what was presented to and approved by the executive council, issued the following law: article (1) • the integrated transport centre shall assume the licensing of the drivers and vehicles as well as all r